<a href="https://colab.research.google.com/github/yogasundaram95/Portfolio-projects/blob/main/Yoga_Sundara_SQL_TakeHome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🚨 **IMPORTANT — READ THIS FIRST**

Before you do anything, **make a private copy of this notebook in your own Google Drive**.

If you edit this shared version, **your work will not be saved** and may be overwritten at any time.

**How to make your copy:**
1. Go to **File → Save a copy in Drive…**
2. Rename it to: `YourName_SQL_TakeHome.ipynb`
3. Work only in *your* copy


In [13]:
# Staffing & Revenue Analytics — SQL + Python Take‑Home

This project analyzes skilled nursing facility staffing and revenue using a mix of SQL (CTEs, joins, aggregations) and Python (data cleaning, record linkage, web scraping, and geospatial visualization). The dataset includes facility metadata, PBJ staffing hours, administrator contact information, synthetic shift‑level revenue data, and sales‑rep deal attribution.[web:1]

## Skills demonstrated
- Advanced SQL: CTEs, windowed logic via grouped aggregates, percentage-of-total calculations, multi-table joins.[web:1]
- Python for analytics: data cleaning, text normalization, record linkage heuristics, and working with SQLite from pandas.[web:1]
- Geospatial visualization: interactive facility map using Plotly Mapbox driven by aggregated PBJ hours.[web:1]
- Web scraping: programmatic discovery and download of a state health department facility file and loading into pandas.[web:1]


SyntaxError: invalid character '‑' (U+2011) (ipython-input-2723667969.py, line 3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 📝 **SQL + Python Take-Home Assessment Instructions**

Welcome! This assessment is designed to evaluate your SQL, Python, and analytical thinking skills.  
Please expand this section and read these instructions carefully before beginning.




### ⏱️ Time Limit

You have **100 minutes** to complete the assessment. We expect the assessment to take no more than 90 minutes, with a 10 minute buffer for submission.

- Your timer began when you submitted the **Start Test** form.
- Your timer ends when you submit the [Finish Test Form](https://docs.google.com/forms/d/e/1FAIpQLSfVXcrAbDRT2KLbNiCtNpDsRSAT63DTUqc5d6wRP-PmrM65uw/viewform?usp=dialog).
- The timestamps on these forms (recorded by Google) determine your official elapsed time.

**If your Finish timestamp is more than 100 minutes after your Start timestamp, your submission will not be accepted.** If you are not able to complete all 3 questions and submit in 100 minutes, please just submit whatever you have completed.

---

### 📄 How to Submit

1. Go to **File → Save a copy in Drive…**  
2. Rename your copy to: YourName_SQL_TakeHome.ipynb
3. Complete the assessment
4. Submit the [Finish Test Form](https://docs.google.com/forms/d/e/1FAIpQLSfVXcrAbDRT2KLbNiCtNpDsRSAT63DTUqc5d6wRP-PmrM65uw/viewform?usp=dialog).

---
### Other Notes

⚠️ **Before answering any questions, scroll down and run the entire “📦 Data Setup & Environment Initialization” section.**

If your runtime resets (common in Colab), you must re-run Setup.


## 📚 **Data Dictionary**

Expand this section to see a reference of all datasets used in this assessment. These tables are created during the Setup phase and loaded into a SQLite in-memory database.



🏥 facilities table

Facility-level metadata for California skilled nursing facilities.

**Columns**

- `cms_id` — CMS Certification Number (primary key, uppercase string)  
- `name` — Facility name  
- `chain_name` — Corporate chain name (may be null)  
- `city` — City  
- `state` — State code ("IL")  
- `zip` — ZIP code (string)  
- `beds` — Number of certified beds  
- `rating` — CMS 5-star rating  
- `latitude` — Latitude  
- `longitude` — Longitude  

**Notes**

- Rows with missing `cms_id` removed  
- Duplicate `cms_id` removed  
- `cms_id` standardized to uppercase  

---

👨‍⚕️ pbj_hours table

Payroll-Based Journal (PBJ) staffing hours for RN, LPN, and CNA roles.

**Columns**

- `id` — Auto-increment primary key  
- `cms_id` — Facility CCN (foreign key to facilities)  
- `work_date` — PBJ entry date (YYYY-MM-DD)  
- `job_title` — RN, LPN, or CNA  
- `total_hours` — Hours worked for that role on that date  

**Notes**

- RN/LPN/CNA columns were melted into long format  
- Only rows with `total_hours > 0` included  
- `cms_id` standardized to uppercase  

---

🧑‍💼 admin_details table

Administrator contact information from state licensure data.

**Columns**

- `facname` — Raw facility name (may differ from CMS naming)  
- `address` — Street address  
- `city` — City  
- `zip` — ZIP code  
- `facadmin` — Administrator name  
- `contact_email` — Administrator email  

**Notes**

- Only active SNFs included  
- Columns normalized to lowercase  
- Some inconsistencies intentionally preserved for matching  

---

🕒 shifts table

Synthetic shift-level operational and financial data.

**Columns**

- `shift_id` — Primary key  
- `cms_id` — Facility CCN  
- `date` — Shift date (YYYY-MM-DD)  
- `specialty` — CNA, LPN, or RN  
- `hours` — Shift length (8 or 12)  
- `pay_rate` — Hourly pay rate  
- `charge_rate` — Hourly bill rate (pay_rate + markup)  

**Notes**

- 5–15 shifts generated per facility  
- `charge_rate` added during Setup  
- Used for revenue and commission calculations  

---

🤝 deals table

Rep attribution metadata for SNF deals.

**Columns**

- `deal_id` — Primary key  
- `cms_id` — Facility CCN  
- `rep_primary` — Primary rep  
- `rep_secondary` — Secondary rep (nullable)  
- `split_primary_pct` — Attribution percentage for primary rep  
- `split_secondary_pct` — Attribution percentage for secondary rep  

**Notes**

- Percentages sum to 1.0  
- Deals may be single-rep or split  

---

🛠️ Helper Function

`run_query(sql_string)` executes an SQL query against the in-memory database and returns a pandas DataFrame.

**Example**

```
run_query("""
SELECT *
FROM facilities
LIMIT 5
""")
```

---


## 📦 **Data Setup & Environment Initialization**

Do not modify the code under this section, but ensure it runs (hit the Run all button at the top under your toolbar before completing anything in the assessment section below)


In [14]:
# ============================================================
# 📘 Test Setup (DO NOT MODIFY THIS CELL)
# ============================================================

!pip install gdown --quiet

import gdown
import sqlite3
import pandas as pd
import random, datetime

# ============================================================
# 🔗 STEP 0 — Download source CSVs using Google Drive file IDs
# ============================================================

SNF_FILE_ID = "1UfCxgMxUtCEDWqcm1udnd7mPawDh7y-b"
PBJ_FILE_ID = "1y9WofLddBZ7ufuAeJ0HEfW9uRlvuQTt7"
ADMIN_FILE_ID = "1mR7vOR3xyeZ6sv4QiclCftOYqB79bajT"

gdown.download(f"https://drive.google.com/uc?id={SNF_FILE_ID}", "IL_SNFs.csv", quiet=False)
gdown.download(f"https://drive.google.com/uc?id={PBJ_FILE_ID}", "IL_PBJ_Hours.csv", quiet=False)
gdown.download(f"https://drive.google.com/uc?id={ADMIN_FILE_ID}", "admin_details_raw.csv", quiet=False)

# ============================================================
# 📁 STEP 1 — Load SNF Facilities dataset
# ============================================================

facilities_df = pd.read_csv("IL_SNFs.csv")
facilities_df = facilities_df.rename(columns={
    "CMS Certification Number (CCN)": "cms_id",
    "Provider Name": "name",
    "City/Town": "city",
    "State": "state",
    "ZIP Code": "zip",
    "Number of Certified Beds": "beds",
    "Overall Rating": "rating",
    "Latitude": "latitude",
    "Longitude": "longitude",
    "Chain Name": "chain_name"
})
facilities_df = facilities_df[
    ["cms_id", "name", "city", "state", "zip", "beds", "rating", "latitude", "longitude", "chain_name"]
].copy()
facilities_df = facilities_df.dropna(subset=["cms_id"]).drop_duplicates(subset=["cms_id"])
facilities_df["cms_id"] = facilities_df["cms_id"].astype(str).str.strip().str.upper()

# ============================================================
# 🏥 STEP 2 — Load PBJ dataset (FIXED: zero-pad CCNs)
# ============================================================

pbj_df = pd.read_csv("IL_PBJ_Hours.csv")
pbj_df.columns = [c.strip().lower() for c in pbj_df.columns]
pbj_df = pbj_df[["provnum", "workdate", "hrs_rn_ctr", "hrs_lpn_ctr", "hrs_cna_ctr"]].copy()

# 🔥 CRITICAL FIX — zero-pad CCNs so they match facilities_df
pbj_df["provnum"] = (
    pbj_df["provnum"]
    .astype(str)
    .str.strip()
    .str.upper()
    .str.zfill(6)        # <—— prevents the top–100 mismatch bug
)

pbj_long_df = pbj_df.melt(
    id_vars=["provnum", "workdate"],
    value_vars=["hrs_rn_ctr", "hrs_lpn_ctr", "hrs_cna_ctr"],
    var_name="job_title",
    value_name="total_hours"
)

pbj_long_df["job_title"] = (
    pbj_long_df["job_title"]
    .str.replace("hrs_", "", regex=False)
    .str.replace("_ctr", "", regex=False)
    .str.upper()
)

pbj_long_df = pbj_long_df.dropna(subset=["total_hours"])
pbj_long_df = pbj_long_df[pbj_long_df["total_hours"] > 0]

pbj_long_df = pbj_long_df.rename(columns={
    "provnum": "cms_id",
    "workdate": "work_date"
})

# ============================================================
# 👤 STEP 3 — Load Admin Details (new dataset)
# ============================================================

admin_details_raw_df = pd.read_csv("admin_details_raw.csv")
admin_details_df = admin_details_raw_df[
    (admin_details_raw_df["FAC_TYPE_CODE"] == "SNF") &
    (admin_details_raw_df["LICENSE_STATUS_DESCRIPTION"] == "ACTIVE")
][["FACNAME", "ADDRESS", "CITY", "ZIP", "FACADMIN", "CONTACT_EMAIL"]].copy()
admin_details_df.columns = [c.lower() for c in admin_details_df.columns]

# ============================================================
# 🗄️ STEP 4 — Create SQLite in-memory DB
# ============================================================

conn = sqlite3.connect(":memory:")
cur = conn.cursor()

cur.executescript("""
DROP TABLE IF EXISTS facilities;
CREATE TABLE facilities (
  cms_id TEXT PRIMARY KEY,
  name TEXT,
  chain_name TEXT,
  city TEXT,
  state TEXT,
  zip TEXT,
  beds INTEGER,
  rating REAL,
  latitude REAL,
  longitude REAL
);

DROP TABLE IF EXISTS pbj_hours;
CREATE TABLE pbj_hours (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  cms_id TEXT,
  work_date TEXT,
  job_title TEXT,
  total_hours REAL,
  FOREIGN KEY(cms_id) REFERENCES facilities(cms_id)
);

DROP TABLE IF EXISTS admin_details;
CREATE TABLE admin_details (
  facname TEXT,
  address TEXT,
  city TEXT,
  zip TEXT,
  facadmin TEXT,
  contact_email TEXT
);
""")

# ============================================================
# 📝 STEP 5 — Insert datasets into SQL
# ============================================================

facilities_df.to_sql("facilities", conn, if_exists="append", index=False)
pbj_long_df.to_sql("pbj_hours", conn, if_exists="append", index=False)
admin_details_df.to_sql("admin_details", conn, if_exists="append", index=False)

# ============================================================
# 🛠️ STEP 6 — Generate deterministic shifts table
# ============================================================

cur.executescript("""
DROP TABLE IF EXISTS shifts;
CREATE TABLE shifts (
  shift_id INTEGER PRIMARY KEY,
  cms_id TEXT,
  date TEXT,
  specialty TEXT,
  hours REAL,
  pay_rate REAL,
  FOREIGN KEY(cms_id) REFERENCES facilities(cms_id)
);
""")

random.seed(42)
specialties = ["CNA", "LPN", "RN"]
hours_options = [8, 12]
date_start = datetime.date(2025, 10, 1)

rows = []
shift_id = 1
for cms_id in facilities_df["cms_id"]:
    for _ in range(random.randint(5, 15)):
        rows.append((
            shift_id,
            cms_id,
            (date_start + datetime.timedelta(days=random.randint(0, 30))).isoformat(),
            random.choice(specialties),
            random.choice(hours_options),
            random.randint(30, 60)
        ))
        shift_id += 1

shifts_df = pd.DataFrame(rows, columns=[
    "shift_id", "cms_id", "date", "specialty", "hours", "pay_rate"
])
shifts_df.to_sql("shifts", conn, if_exists="append", index=False)

# ============================================================
# ⚡ STEP 7 — SQL helper
# ============================================================

def run_query(query: str):
    return pd.read_sql_query(query, conn)

# ============================================================
# 🆕 STEP 8 — Generate Deals Table
# ============================================================

shifts_df["charge_rate"] = shifts_df["pay_rate"] + shifts_df["pay_rate"].apply(
    lambda x: random.randint(15, 40)
)
cur.executescript("DROP TABLE IF EXISTS shifts;")
shifts_df.to_sql("shifts", conn, index=False, if_exists="replace")

cur.executescript("""
DROP TABLE IF EXISTS deals;
CREATE TABLE deals (
  deal_id INTEGER PRIMARY KEY,
  cms_id TEXT,
  rep_primary TEXT,
  rep_secondary TEXT,
  split_primary_pct REAL,
  split_secondary_pct REAL
);
""")

reps = ["Alex", "Taylor", "Jordan", "Morgan", "Casey"]
sample_facilities = random.sample(list(facilities_df["cms_id"]), 10)

deal_rows = []
deal_id = 1
for cms in sample_facilities:
    rep1 = random.choice(reps)
    if random.random() < 0.5:
        rep2 = random.choice([r for r in reps if r != rep1])
        p1, p2 = random.choice([(0.5, 0.5), (0.75, 0.25)])
    else:
        rep2 = None
        p1, p2 = 1.0, 0.0
    deal_rows.append((deal_id, cms, rep1, rep2, p1, p2))
    deal_id += 1

deal_df = pd.DataFrame(deal_rows, columns=[
    "deal_id", "cms_id", "rep_primary", "rep_secondary",
    "split_primary_pct", "split_secondary_pct"
])
deal_df.to_sql("deals", conn, index=False, if_exists="append")

print("Setup complete.")

Downloading...
From: https://drive.google.com/uc?id=1UfCxgMxUtCEDWqcm1udnd7mPawDh7y-b
To: /content/IL_SNFs.csv
100%|██████████| 726k/726k [00:00<00:00, 77.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1y9WofLddBZ7ufuAeJ0HEfW9uRlvuQTt7
To: /content/IL_PBJ_Hours.csv
100%|██████████| 17.1M/17.1M [00:00<00:00, 95.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mR7vOR3xyeZ6sv4QiclCftOYqB79bajT
To: /content/admin_details_raw.csv
100%|██████████| 7.42M/7.42M [00:00<00:00, 309MB/s]
/tmp/ipython-input-149852213.py:51: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



Setup complete.


## 🧪 **Assessment**

See below each of the 3 questions, with boxes for you to input your solution to each one.

📌 **Question 1 — PBJ Hours for the Top 10 Chains**

Create a SQL query that returns PBJ staffing activity for the **top 10 chains** in the dataset.

Your output **must include the following columns:**

- **chain_name** – Name of the chain  
- **total_facilities** – Total number of facilities that belong to that chain  
- **facilities_with_pbj_hours** – Number of chain facilities that reported > 0 hours in PBJ  
- **total_agency_hours** – Total CNA + LPN + RN hours for all facilities within that chain  
- **pct_of_statewide_hours** – Percentage of all PBJ hours in the state represented by this chain  

---

**Rules & Notes**

- Include all worker types (**RN, LPN, CNA**).  
- Only include PBJ entries with `total_hours > 0`.  
- Only include chains that appear in the **top 10 by total agency hours**.  
- Do **not** include rows where `chain_name` is NULL or blank.  
- Sort the result by **total_agency_hours DESC**.  


In [15]:
# QUESTION 1
# Write your SQL query below
# Reminder that SQL must be wrapped in run_query() - example below

run_query("""

-- Calculate stats for each chain
WITH chain_stats AS (
  SELECT
    f.chain_name,
    COUNT(DISTINCT f.cms_id) AS total_facilities,
    COUNT(DISTINCT p.cms_id) AS facilities_with_pbj_hours,
    SUM(p.total_hours) AS total_agency_hours
  FROM facilities f
  LEFT JOIN pbj_hours p ON f.cms_id = p.cms_id
  WHERE f.chain_name IS NOT NULL
    AND f.chain_name != ''
  GROUP BY f.chain_name
),

-- Gets the total hours across all facilities in Illinois
statewide AS (
  SELECT SUM(total_hours) AS total
  FROM pbj_hours
)

-- Show top 10 chains with their percentage of state hours
SELECT
  chain_name,
  total_facilities,
  facilities_with_pbj_hours,
  total_agency_hours,
  ROUND(total_agency_hours * 100.0 / statewide.total, 2) AS pct_of_statewide_hours
FROM chain_stats, statewide
ORDER BY total_agency_hours DESC
LIMIT 10

""")

,chain_name,total_facilities,facilities_with_pbj_hours,total_agency_hours,pct_of_statewide_hours
0,PACS GROUP,125,84,172739.87,14.98
1,MARINER HEALTH CARE,18,17,160999.32,13.97
2,ASPEN SKILLED HEALTHCARE,30,22,49993.21,4.34
3,BRIUS MANAGEMENT,36,18,47877.09,4.15
4,ROLLINS-NELSON HEALTHCARE MANAGEMENT,10,7,45537.63,3.95
5,GOLDEN SNF OPERATIONS,7,7,44516.85,3.86
6,THE ENSIGN GROUP,69,34,39047.36,3.39
7,EVA CARE GROUP,9,8,31544.50,2.74
8,PROMEDICA SENIOR CARE,6,5,17417.54,1.51
9,LINKS HEALTHCARE GROUP,21,11,16445.25,1.43


### 📌 **Question 2 — Match Admin Contacts for the Top 100 Facilities (Python)**

Using the `admin_details` dataset and the `facilities` dataset, create a Python script that matches **administrator contact information for the top 100 facilities by PBJ hours**.

### **Requirements**

1. **Identify the top 100 facilities by total PBJ hours**
   - Use the `pbj_hours` table.

2. **Match admin contacts for those 100 facilities**
   - Use the 'admin_details' table

3. **Final output must include one row per facility**  
   Columns:
   - `cms_id`
   - `facility_name`
   - `address`
   - `city`
   - `zip`
   - `admin_name`
   - `admin_email`

---

### **Goal**
Return **as many matched administrator contacts as possible** for the top 100 facilities.


In [20]:
# Save final_output to CSV
final_output.to_csv('q2_admin_matches.csv', index=False)
print('Saved q2_admin_matches.csv')

Saved q2_admin_matches.csv


In [16]:
# QUESTION 2
# Write your Python code below.


# Your code here
# QUESTION 2
# Match admin contacts to top 100 facilities by staffing hours

# Get the top 100 facilities
top_100 = run_query("""
SELECT f.cms_id, f.name AS facility_name, f.city, f.zip, SUM(p.total_hours) AS total_hours
FROM facilities f
JOIN pbj_hours p ON f.cms_id = p.cms_id
GROUP BY f.cms_id, f.name, f.city, f.zip
ORDER BY total_hours DESC
LIMIT 100
""")

# Get admin details
admins = run_query("""
SELECT facname, address, city, zip, facadmin AS admin_name, contact_email AS admin_email
FROM admin_details
""")

# Step 3: Clean the data for better matching
def clean_text(text):
    """Remove punctuation and standardize text"""
    if pd.isna(text):
        return ''
    return str(text).upper().strip().replace(',', '').replace('.', '').replace('-', '')

def clean_zip(zip_code):
    """Get first 5 digits of ZIP"""
    return str(zip_code).strip()[:5]

# Apply cleaning
top_100['name_clean'] = top_100['facility_name'].apply(clean_text)
top_100['city_clean'] = top_100['city'].apply(clean_text)
top_100['zip_clean'] = top_100['zip'].apply(clean_zip)

admins['name_clean'] = admins['facname'].apply(clean_text)
admins['city_clean'] = admins['city'].apply(clean_text)
admins['zip_clean'] = admins['zip'].apply(clean_zip)

# Step 4: Try matching strategies
matched = top_100.copy()
matched['address'] = None
matched['admin_name'] = None
matched['admin_email'] = None

# Strategy 1: Match on facility name + ZIP
for idx, facility in matched.iterrows():
    # Try exact name + ZIP match first
    match = admins[
        (admins['name_clean'] == facility['name_clean']) &
        (admins['zip_clean'] == facility['zip_clean'])
    ]

    if len(match) > 0:
        matched.loc[idx, 'address'] = match.iloc[0]['address']
        matched.loc[idx, 'admin_name'] = match.iloc[0]['admin_name']
        matched.loc[idx, 'admin_email'] = match.iloc[0]['admin_email']
        continue

    # Strategy 2: If no match, try city + ZIP with partial name matching
    candidates = admins[
        (admins['city_clean'] == facility['city_clean']) &
        (admins['zip_clean'] == facility['zip_clean'])
    ]

    if len(candidates) > 0:
        # Find candidate with most words in common
        facility_words = set(facility['name_clean'].split())
        best_match = None
        best_score = 0

        for _, candidate in candidates.iterrows():
            candidate_words = set(candidate['name_clean'].split())
            common = len(facility_words & candidate_words)

            if common > best_score:
                best_score = common
                best_match = candidate

        # Only match if we have at least 2 words in common
        if best_match is not None and best_score >= 2:
            matched.loc[idx, 'address'] = best_match['address']
            matched.loc[idx, 'admin_name'] = best_match['admin_name']
            matched.loc[idx, 'admin_email'] = best_match['admin_email']

# Create final output
final_output = matched[[
    'cms_id',
    'facility_name',
    'address',
    'city',
    'zip',
    'admin_name',
    'admin_email'
]].copy()

# Show results
matched_count = final_output['admin_email'].notna().sum()
print(f"Matched {matched_count} out of 100 facilities ({matched_count}%)")
print()

final_output

Matched 94 out of 100 facilities (94%)



,cms_id,facility_name,address,city,zip,admin_name,admin_email
0,056389,VALE HEALTHCARE CENTER,13484 SAN PABLO AVENUE,SAN PABLO,94806,"NOYES, DEBORAH",P.CHIB@MARINERHEALTHCARE.COM
1,555431,COUNTRY HILLS POST ACUTE,1580 BROADWAY,EL CAJON,92021,"GRAF, ZACHARY",ZACH.GRAF@COUNTRYHILLSPA.COM
2,555766,SIERRA VIEW MEDICAL CENTER,465 W PUTNAM AVE,PORTERVILLE,93257,"PICHAY, CERELINA",None
3,055318,SKYLINE HEALTHCARE CENTER - SAN JOSE,2065 FOREST AVE,SAN JOSE,95128,"PENYACSEK, MARK",MARK.PENYACSEK@MARINERHEALTHCARE.COM
4,056272,SAN FRANCISCO HEALTH CARE,1477 GROVE ST,SAN FRANCISCO,94117,None,GENERALCONTACT@SFHCR.COM
...,...,...,...,...,...,...,...
95,055136,BERKLEY WEST HEALTHCARE CENTER,1623 ARIZONA AVE,SANTA MONICA,90404,"YANG, ANDREW",ANDREWY@ASPENHEALTH.COM
96,555433,EASTERN PLUMAS HOSPITAL- PORTOLA CAMPUS DP/SNF,500 1ST STREET,PORTOLA,96122,"HAYES, THOMAS P",TSANTELLA@EPHC.ORG
97,055239,EAST BAY POST-ACUTE,20259 LAKE CHABOT ROAD,CASTRO VALLEY,94546,"LEGAS, GUNNAR A",None
98,555076,STILLWATER POST-ACUTE,510 E WASHINGTON AVE,EL CAJON,92020,"COMRIE, MARY",MARY.COMRIE@LINKSHEALTH.COM


### 📌 **Question 3 — Deal Revenue & Commission Attribution (SQL)**

You are given a new table called `deals` that attributes facilities to one or two sales reps who signed them.
Each deal may be:

- fully attributed to one rep, or  
- split between two reps (**75% / 25%** or **50% / 50%**)

You will also use the `shifts` table, where each shift includes:

- `hours`  
- `pay_rate`  
- `charge_rate`

Revenue from a shift is calculated as:

revenue = (charge_rate - pay_rate) * hours


---

### **Requirements**

1. **Determine the 30-day revenue window for each deal**
   - For each `cms_id` in `deals`, find the **first shift date** in the `shifts` table.
   - Include all shifts occurring **within 30 days after** that first shift date.

2. **Calculate total revenue for each deal**
   - Use the revenue formula above.
   - Sum all revenue across all qualifying shifts for each deal.

3. **Allocate commissions to sales reps**
   - Each rep earns **20% of attributed revenue**, allocated by the deal’s split percentages:
     ```
     commission = revenue * 0.20 * rep_split_pct
     ```
   - `rep_split_pct` will be:
     - `1.0` for a fully owned deal  
     - `0.75` or `0.25` for 75/25 splits  
     - `0.50` for equal splits  

4. **Final output must return one row per rep per deal**  
   Columns:
   - `deal_id`  
   - `cms_id`  
   - `rep_name`  
   - `rep_split_pct`  
   - `total_revenue`  
   - `commission_owed`

5. **Sort the final output**
   - First by `commission_owed` **DESC**  
   - Then by `rep_name` **ASC**

---

### **Goal**

Produce a result showing how much **revenue** each deal generated and how much **commission** each rep earns based on the predefined attribution splits.



In [17]:
# QUESTION 3
# Write your SQL query below
# Reminder that SQL must be wrapped in run_query() - example below

# QUESTION 3
# Calculate revenue and commissions for sales rep deals

run_query("""

-- Find the first shift date for each deal
WITH first_shift AS ( SELECT d.deal_id, d.cms_id, MIN(s.date) AS start_date
  FROM deals d
  JOIN shifts s ON d.cms_id = s.cms_id
  GROUP BY d.deal_id, d.cms_id
),

-- Calculate total revenue from shifts in the 30-day window
revenue_30days AS ( SELECT f.deal_id, f.cms_id, SUM((s.charge_rate - s.pay_rate) * s.hours) AS total_revenue
  FROM first_shift f
  JOIN shifts s ON f.cms_id = s.cms_id
  WHERE s.date BETWEEN f.start_date AND DATE(f.start_date, '+30 days')
  GROUP BY f.deal_id, f.cms_id
)

-- Split revenue across reps and calculate their commissions
SELECT d.deal_id, d.cms_id, d.rep_primary AS rep_name, d.split_primary_pct AS rep_split_pct, ROUND(r.total_revenue, 2) AS total_revenue, ROUND(r.total_revenue * 0.20 * d.split_primary_pct, 2) AS commission_owed
FROM deals d
JOIN revenue_30days r ON d.deal_id = r.deal_id

UNION ALL

SELECT d.deal_id, d.cms_id, d.rep_secondary AS rep_name, d.split_secondary_pct AS rep_split_pct, ROUND(r.total_revenue, 2) AS total_revenue, ROUND(r.total_revenue * 0.20 * d.split_secondary_pct, 2) AS commission_owed
FROM deals d
JOIN revenue_30days r ON d.deal_id = r.deal_id
WHERE d.rep_secondary IS NOT NULL

ORDER BY commission_owed DESC, rep_name ASC

""")

,deal_id,cms_id,rep_name,rep_split_pct,total_revenue,commission_owed
0,8,555638,Morgan,1.00,4136.0,827.2
1,9,555923,Casey,1.00,3636.0,727.2
2,7,555200,Alex,1.00,3572.0,714.4
3,5,055008,Taylor,1.00,3500.0,700.0
4,4,055350,Jordan,1.00,3448.0,689.6
5,10,055462,Morgan,1.00,2604.0,520.8
6,2,056425,Casey,1.00,2540.0,508.0
7,3,555796,Taylor,0.75,2768.0,415.2
8,6,555116,Taylor,0.75,2408.0,361.2
9,1,056008,Jordan,1.00,1452.0,290.4


## 🔥 **Extra Credit**

You will not be penalized at all if you leave this section blank, but if you complete the first 3 questions with time to spare here is another couple you can work on to showcase additional skills.


### 🔥 Extra Credit #1 — Interactive Facility Map

Using the facilities table and the pbj_hours table, create an **interactive map** that visualizes all facilities in the dataset.

Your map should include:

- **One point per facility**
- **Dot size proportional to total PBJ hours** (RN + LPN + CNA)
- **Hover tooltip** that displays:
  - Facility name  
  - Address (city + zip is acceptable)
  - Total PBJ Hours

**Again:** This question is completely optional and will not impact your score for the main assessment. It is an opportunity to showcase your Python + visualization skills if you finish early.

In [18]:
# EXTRA CREDIT #1 - Interactive Facility Map

import plotly.express as px

# Get facilities with PBJ hours
data = run_query("""
SELECT
  f.name,
  f.city,
  f.zip,
  f.latitude AS lat,
  f.longitude AS lon,
  COALESCE(SUM(p.total_hours), 0) AS hours
FROM facilities f
LEFT JOIN pbj_hours p ON f.cms_id = p.cms_id
WHERE f.latitude IS NOT NULL
GROUP BY f.name, f.city, f.zip, f.latitude, f.longitude
""")

# Create map
fig = px.scatter_mapbox(
    data,
    lat='lat',
    lon='lon',
    size='hours',
    color='hours',
    hover_name='name',
    hover_data={'city': True, 'zip': True, 'hours': ':,.0f', 'lat': False, 'lon': False},
    color_continuous_scale='Viridis',
    size_max=20,
    zoom=6,
    title='Illinois SNFs - PBJ Staffing Hours',
    mapbox_style='open-street-map',
    height=700
)

fig.show()

### 🔥 Extra Credit #2 — Adult Care Home CSV Scraper

Using Python, scrape the **Adult Care Home CSV** from the NC Department of Health and Human Services website:

🔗 https://info.ncdhhs.gov/dhsr/acls/faclistings.html

Your task:

1. Programmatically locate the **Adult Care Home CSV** link on that page.
2. Download the CSV file.
3. Load it into a **pandas DataFrame**.
4. Display the first 10 rows.

**Again:** This question is completely optional and will not impact your score for the main assessment. It is an opportunity to showcase your Python skills if you finish early.


In [19]:
# EXTRA CREDIT #2 - Adult Care Home File Scraper

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Get the webpage
url = 'https://info.ncdhhs.gov/dhsr/acls/faclistings.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Step 2: Find Adult Care Home XLSX file
data_link = None
for link in soup.find_all('a', href=True):
    if 'ahlist.xlsx' in link['href'].lower():
        data_link = link['href']
        break

# Step 3: Build full URL and download
if data_link.startswith('../'):
    full_url = 'https://info.ncdhhs.gov/dhsr/' + data_link.replace('../', '')
else:
    full_url = 'https://info.ncdhhs.gov/dhsr/acls/' + data_link

print(f"Downloading: {full_url}\n")

# Step 4: Load XLSX with proper headers (skip title rows)
file_data = requests.get(full_url)
df = pd.read_excel(pd.io.common.BytesIO(file_data.content), skiprows=2)

# Step 5: Display results
print(f"✓ Loaded {len(df)} Adult Care Homes")
print(f"✓ Columns: {list(df.columns[:5])}...\n")
df.head(10)

Downloading: https://info.ncdhhs.gov/dhsr/data/Ahlist.xlsx?ver=2.7

✓ Loaded 575 Adult Care Homes
✓ Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Licensed by the State of North Carolina\nDepartment of Health and Human Services - Division of Health Service Regulation']...



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Licensed by the State of North Carolina\nDepartment of Health and Human Services - Division of Health Service Regulation,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,As of 11/2025,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RowNo.,License #,Name of Licensee Legal Name,NaN,NaN,NaN,DBA Name,NaN,Correspondence Name,NaN,...,Site Zip,Facility Address,Facility Address 2,Facility City,Facility State,Facility Zip,County,Bed Count,Star Rating,Expiry Date
4,1,HAL-001-173,Burlington NC SLC Tenant LLC,NaN,NaN,NaN,Homeplace of Burlington,NaN,Sarah Gabel,NaN,...,27215,118 Alamance Road,NaN,Burlington,NC,27215,Alamance,67,2,31-Dec-25
5,2,HAL-001-172,A Vision Come True Assistant Living Facility LLC,NaN,NaN,NaN,A Vision Come True,NaN,Tammie Staton,NaN,...,27217,P.O. Box 35,NaN,Burlington,NC,27217,Alamance,12,3,31-Dec-25
6,3,HAL-001-171,Clegail Professional Services LLC,NaN,NaN,NaN,Cape Point Memory Care Unit,NaN,CLEMENT SOWA,NaN,...,27215,5635 HORNADAY RD UNIT D,NaN,GREENSBORO,NC,27409,Alamance,12,0,31-Dec-25
7,4,HAL-001-170,Clegail Professional Services LLC,NaN,NaN,NaN,Emory Adult Care Home,NaN,CLEMENT SOWA,NaN,...,27215,5635 HORNADAY RD UNIT D,NaN,GREENSBORO,NC,27409,Alamance,12,3,31-Dec-25
8,5,HAL-001-169,"Springview Senior Living, Inc",NaN,NaN,NaN,SpringView-Cook Building,NaN,Beverly McHugh,NaN,...,27244,PO Box 2175,NaN,Burlington,NC,27216,Alamance,12,3,31-Dec-25
9,6,HAL-001-167,Making Visions Come True Assistant Living Faci...,NaN,NaN,NaN,Making Visions Come True Assistant Living Fac...,NaN,Tammie Wright Staton,NaN,...,27217,PO Box 35,NaN,Burlington,NC,27217,Alamance,12,3,31-Dec-25
